# Competitive Analysis Demo

This notebook demonstrates the competitive analysis modules for MYGA products:

1. **Positioning Analysis** - Rate percentiles and distribution
2. **Spread Analysis** - Spreads over Treasury
3. **Company Rankings** - Market rankings and competitive landscape

See: CONSTITUTION.md Section 5

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import competitive analysis modules
from annuity_pricing.competitive import (
    PositioningAnalyzer,
    SpreadAnalyzer,
    RankingAnalyzer,
    build_treasury_curve,
)
from annuity_pricing.products.myga import MYGAPricer
from annuity_pricing.data.schemas import MYGAProduct

# Set display options
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 120)

## 1. Generate Sample Market Data

For demonstration, we'll create synthetic market data that mimics WINK patterns.

[T2] Based on typical WINK MYGA distributions:
- 3-year rates: ~4.0-4.5%
- 5-year rates: ~4.3-5.0%
- 7-year rates: ~4.5-5.2%

In [ ]:
# Seed for reproducibility
np.random.seed(42)

companies = [
    "Alpha Life", "Beta Insurance", "Gamma Annuity", "Delta Financial",
    "Epsilon Life", "Zeta Insurance", "Eta Annuity", "Theta Financial",
    "Iota Life", "Kappa Insurance"
]

def generate_market_data(n_products=100):
    """Generate synthetic MYGA market data."""
    data = []
    
    for i in range(n_products):
        duration = np.random.choice([3, 5, 7], p=[0.25, 0.50, 0.25])
        
        # Rate depends on duration (longer = higher)
        base_rate = 0.040 + (duration - 3) * 0.003
        rate = np.random.normal(base_rate, 0.004)
        rate = max(0.030, min(0.060, rate))  # Clip to reasonable bounds
        
        data.append({
            'companyName': np.random.choice(companies),
            'productName': f'{duration}-Year MYGA {i+1}',
            'productGroup': 'MYGA',
            'status': 'current',
            'fixedRate': round(rate, 4),
            'guaranteeDuration': duration,
        })
    
    return pd.DataFrame(data)

market_data = generate_market_data(150)
print(f"Generated {len(market_data)} MYGA products")
market_data.head(10)

## 2. Positioning Analysis

Analyze where a product's rate falls within the competitive landscape.

In [ ]:
# Initialize analyzer
positioning = PositioningAnalyzer()

# Analyze a specific rate
target_rate = 0.045  # 4.5%

# Position for 5-year MYGA
position = positioning.analyze_position(
    rate=target_rate,
    market_data=market_data,
    product_group='MYGA',
    guarantee_duration=5,
)

print("=" * 50)
print("POSITIONING ANALYSIS")
print("=" * 50)
print(f"Rate: {target_rate:.2%}")
print(f"Percentile: {position.percentile:.1f}")
print(f"Rank: {position.rank} of {position.total_products}")
print(f"Quartile: {position.quartile}")
print(f"Position: {position.position_label}")

In [ ]:
# Get distribution statistics
stats = positioning.get_distribution_stats(
    market_data=market_data,
    product_group='MYGA',
    guarantee_duration=5,
)

print("\n" + "=" * 50)
print("5-YEAR MYGA RATE DISTRIBUTION")
print("=" * 50)
print(f"Count: {stats.count}")
print(f"Min: {stats.min:.3%}")
print(f"Max: {stats.max:.3%}")
print(f"Mean: {stats.mean:.3%}")
print(f"Median: {stats.median:.3%}")
print(f"Std Dev: {stats.std:.4%}")
print(f"Q1 (25th): {stats.q1:.3%}")
print(f"Q3 (75th): {stats.q3:.3%}")

In [ ]:
# Visualize rate distribution
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for idx, duration in enumerate([3, 5, 7]):
    duration_data = market_data[
        (market_data['guaranteeDuration'] == duration) &
        (market_data['productGroup'] == 'MYGA')
    ]['fixedRate']
    
    ax = axes[idx]
    ax.hist(duration_data * 100, bins=15, edgecolor='black', alpha=0.7)
    ax.axvline(duration_data.median() * 100, color='red', linestyle='--', label='Median')
    ax.set_xlabel('Rate (%)')
    ax.set_ylabel('Count')
    ax.set_title(f'{duration}-Year MYGA Rate Distribution')
    ax.legend()

plt.tight_layout()
plt.show()

## 3. Spread Analysis

Analyze spreads over Treasury yields.

In [ ]:
# Build Treasury curve (sample rates)
treasury_curve = {
    1: 0.035,   # 1-year: 3.5%
    2: 0.037,   # 2-year: 3.7%
    3: 0.038,   # 3-year: 3.8%
    5: 0.040,   # 5-year: 4.0%
    7: 0.042,   # 7-year: 4.2%
    10: 0.044,  # 10-year: 4.4%
}

# Initialize spread analyzer
spread_analyzer = SpreadAnalyzer()

# Calculate spread for a specific product
product_rate = 0.045
duration = 5

spread_result = spread_analyzer.calculate_spread(
    product_rate=product_rate,
    treasury_rate=treasury_curve[duration],
    duration=duration,
)

print("=" * 50)
print("SPREAD ANALYSIS")
print("=" * 50)
print(f"Product Rate: {spread_result.product_rate:.3%}")
print(f"Treasury Rate ({duration}Y): {spread_result.treasury_rate:.3%}")
print(f"Spread: {spread_result.spread_bps:.0f} bps ({spread_result.spread_pct:.1f}% over Treasury)")

In [ ]:
# Calculate spreads for all market products
market_with_spreads = spread_analyzer.calculate_market_spreads(
    market_data=market_data,
    treasury_curve=treasury_curve,
    product_group='MYGA',
)

# Get spread distribution for 5-year products
spread_dist = spread_analyzer.get_spread_distribution(
    market_data=market_data,
    treasury_curve=treasury_curve,
    product_group='MYGA',
    guarantee_duration=5,
)

print("\n" + "=" * 50)
print("5-YEAR MYGA SPREAD DISTRIBUTION")
print("=" * 50)
print(f"Count: {spread_dist.count}")
print(f"Min Spread: {spread_dist.min_bps:.0f} bps")
print(f"Max Spread: {spread_dist.max_bps:.0f} bps")
print(f"Mean Spread: {spread_dist.mean_bps:.0f} bps")
print(f"Median Spread: {spread_dist.median_bps:.0f} bps")
print(f"Std Dev: {spread_dist.std_bps:.1f} bps")

In [ ]:
# Spread by duration analysis
spread_by_duration = spread_analyzer.spread_by_duration(
    market_data=market_data,
    treasury_curve=treasury_curve,
    product_group='MYGA',
)

print("\n" + "=" * 50)
print("SPREAD BY DURATION")
print("=" * 50)
print(spread_by_duration.to_string(index=False))

## 4. Company Rankings

Analyze company competitive positioning and market rankings.

In [ ]:
# Initialize ranking analyzer
ranking_analyzer = RankingAnalyzer()

# Get market summary
summary = ranking_analyzer.market_summary(market_data, product_group='MYGA')

print("=" * 50)
print("MARKET SUMMARY")
print("=" * 50)
print(f"Total Products: {summary['total_products']}")
print(f"Total Companies: {summary['total_companies']}")
print(f"Rate Range: {summary['rate_min']:.3%} - {summary['rate_max']:.3%}")
print(f"Mean Rate: {summary['rate_mean']:.3%}")
print(f"Median Rate: {summary['rate_median']:.3%}")
print(f"Durations Available: {summary['durations_available']}")

In [ ]:
# Rank companies by best rate (5-year products)
company_rankings = ranking_analyzer.rank_companies(
    market_data=market_data,
    product_group='MYGA',
    guarantee_duration=5,
    rank_by='best_rate',
)

print("\n" + "=" * 50)
print("COMPANY RANKINGS (5-Year MYGA, by Best Rate)")
print("=" * 50)
print(f"{'Rank':<6}{'Company':<20}{'Best Rate':<12}{'Avg Rate':<12}{'Products':<10}")
print("-" * 60)

for r in company_rankings[:10]:
    print(f"{r.rank:<6}{r.company:<20}{r.best_rate:.3%}       {r.avg_rate:.3%}       {r.product_count}")

In [ ]:
# Rate leaders by duration
leaders = ranking_analyzer.rate_leaders_by_duration(
    market_data=market_data,
    product_group='MYGA',
)

print("\n" + "=" * 50)
print("RATE LEADERS BY DURATION")
print("=" * 50)

for duration in sorted(leaders['duration'].unique()):
    print(f"\n{duration}-Year MYGA:")
    duration_leaders = leaders[leaders['duration'] == duration]
    for _, row in duration_leaders.iterrows():
        print(f"  #{int(row['rank'])}: {row['company']:<18} {row['rate']:.3%}")

In [ ]:
# Competitive landscape
landscape = ranking_analyzer.competitive_landscape(
    market_data=market_data,
    product_group='MYGA',
    guarantee_duration=5,
)

print("\n" + "=" * 50)
print("COMPETITIVE LANDSCAPE (5-Year MYGA)")
print("=" * 50)
print(landscape[['companyName', 'rank', 'best_rate', 'avg_rate', 'tier', 'rate_percentile']].to_string(index=False))

## 5. Rate Recommendation

Use the rate recommender to suggest competitive rates.

In [ ]:
from annuity_pricing.rate_setting.recommender import RateRecommender

recommender = RateRecommender()

# Get rate recommendations for different target percentiles
print("=" * 60)
print("RATE RECOMMENDATIONS (5-Year MYGA)")
print("=" * 60)
print(f"{'Target Percentile':<20}{'Recommended Rate':<18}{'Spread (bps)':<15}")
print("-" * 55)

for target in [50, 75, 90, 95]:
    rec = recommender.recommend_rate(
        guarantee_duration=5,
        target_percentile=target,
        market_data=market_data,
        treasury_rate=treasury_curve[5],
    )
    spread = rec.spread_over_treasury or 0
    print(f"{target}th percentile      {rec.recommended_rate:.3%}           {spread:.0f}")

In [ ]:
# Sensitivity analysis
sensitivity = recommender.sensitivity_analysis(
    guarantee_duration=5,
    market_data=market_data,
    treasury_rate=treasury_curve[5],
    percentile_range=[25, 50, 75, 90],
)

print("\n" + "=" * 60)
print("SENSITIVITY ANALYSIS")
print("=" * 60)
print(sensitivity[['percentile', 'rate', 'spread_bps', 'margin_bps']].to_string(index=False))

## Summary

This notebook demonstrated the competitive analysis modules:

1. **PositioningAnalyzer**: Rate percentile analysis and distribution statistics
2. **SpreadAnalyzer**: Spread over Treasury calculations and market spread distributions
3. **RankingAnalyzer**: Company rankings, market summary, and competitive landscape
4. **RateRecommender**: Rate recommendations for target competitive percentiles

### Key Insights

- **Positioning**: Understand where a product rate falls in the competitive landscape
- **Spreads**: Track spread over Treasury to assess margin and competitiveness
- **Rankings**: Monitor competitive positioning relative to peers
- **Recommendations**: Set rates to achieve target competitive percentiles